First things first: I have some data analysis background from my research career in particle physics, but I am far from being an machine learning expert. So please bear with me and I am happy to receive any kind of feedback.

Since the training data set (and possibly the test data as well) contain missing data, I wanted to have a closer look at this issue. I have seen that other participants propose to fill those NaNs with the mean or median for the respective column. Here I am not (yet) that much interested in filling the blanks but I rather want to know whether we can learn something more about the data when looking missing values.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
with pd.HDFStore('train.h5') as train:
    df = train.get('train')

Let's see how much data we've got:

In [ ]:
print(df.shape)

This is very interesting. This means that rows with missing data for a given ID and feature are always continuous in time. It is **not** the case that data for an asset is unavailable, becomes available and then becomes unavailable again. I bet that if data is unavailable, it is at the beginning of the time interval in which the corresponding asset is traded. Let's try to visualize this.

In [ ]:
# iterate over IDs
grouped = df.groupby('id');
for i,(n,g) in enumerate(grouped):
    # get missing data flag for feature columns
    d = g.isnull().drop(['timestamp','id','y'],axis=1)
    # normalise time stamp to start with 0 when ID appears for the first time in portfolio
    d.index = g.timestamp - g.timestamp.min()
    d.index.name = 'relative timestamp'
    plt.figure(figsize=(16,12))
    plt.title("ID = %d" % n)
    sns.heatmap(d.T,xticklabels=100,cbar=False)
    # only plot first 10 IDs
    if i > 10:
        break

Bingo! Black bars denote columns and time stamps with missing data. All bars start at the very left which is the time an ID appears for the first time in the portfolio. It also looks like there are different *kinds* of these _barcode plots_. Maybe one can try to categorise IDs based on their missing value patterns... I will post an update on this.

In [ ]:
ids = sorted(df.id.unique())
columns = df.columns.drop(['id','timestamp','y']).insert(0,'size')
nan_df = pd.DataFrame(data=None,index=ids,columns=columns,dtype=float)
# iterate over all asset ID
for name,group in df.groupby('id'):
    # for every feature column
    for c in columns:
        if c == 'size':
            nan_df.loc[name,c] = int(len(group))
        else:
            # total number of rows with missing data
            nan_df.loc[name,c] = float(group[c].isnull().sum())

In [ ]:
nan_df.head()

In [ ]:
binary = nan_df.drop(['size'],axis=1)
binary[binary > 0] = 1
binary.head()

In [ ]:
from sklearn.cluster import dbscan
_,labels = dbscan(binary.values,eps=3,min_samples=10)
plt.hist(labels);

In [ ]:
fractional = nan_df.div(nan_df['size'],axis='index').drop(['size'],axis=1)
fractional.head()

In [ ]:
_,labels = dbscan(fractional.values,min_samples=10)
plt.hist(labels,range=(-1.5,labels.max()+0.5),bins=labels.max()+2);

In [ ]:
fractional2 = nan_df.copy()
fractional2[fractional2 > 100] = 100
fractional2['size'] = nan_df['size']
fractional2 = fractional2.div(fractional2['size'],axis='index').drop(['size'],axis=1)
fractional2.head()

In [ ]:
_,labels = dbscan(fractional2.values,eps=2,min_samples=10)
plt.hist(labels,range=(-1.5,labels.max()+0.5),bins=labels.max()+2);

In [ ]:
fractional2.describe()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#Range for k
kmin = 2
kmax = 25
sil_scores = []

#Compute silouhette scoeres
for k in range(kmin,kmax):
    km = KMeans(n_clusters=k).fit(fractional.values)
    sil_scores.append(silhouette_score(fractional.values, km.labels_))

#Plot
plt.plot(range(kmin,kmax), sil_scores)
plt.title('KMeans Results')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()
#_,labels,_ = k_means(fractional.values,n_clusters=10)
#plt.hist(labels,range=(-1.5,labels.max()+0.5),bins=labels.max()+2);

In [ ]:
t = df.groupby('timestamp').apply(lambda x: x.isnull().sum())

In [ ]:
t.head()

In [ ]:
a = (t - t.shift()).drop(['id','timestamp','y'],axis=1)

In [ ]:
n = df.groupby('timestamp').apply(len)
b = (n - n.shift())

In [ ]:
a.plot(figsize=(24,12))
b.plot(style=['.b'],ax=plt.gca())

In [ ]:
a[:350].plot(figsize=(16,8))
b[:350].plot(style=['.b'],ax=plt.gca())

In [ ]:
max_lag = 250
corrs = np.zeros((max_lag,a.shape[1]))
for l in range(1,max_lag):
    c = a.shift(-l)
    corrs[l] = c.corrwith(b)

In [ ]:
plt.figure(figsize=(30,18))
sns.heatmap(corrs,xticklabels=a.columns.tolist())

In [ ]:
a.shift(6).corrwith(b).filter(regex="technical_*")